# Migration script

Migrate the input databases from the 2.29 format to the 2.31.1 format.

This notebook assumes `SCENARIO_TO_MIGRATE` points to a scenario of Kleinalbis (which buildings?) in v2.29.0 format and `EXAMPLE_SCENARIO` to a scenario in v2.31.1 format. 

These are the v2.29.0 format files to migrate:

- inputs/building-geometry/zone.shp
- inputs/building-geometry/site.shp
- inputs/building-geometry/surroundings.shp

- inputs/building-properties/age.dbf
- inputs/building-properties/air_conditioning_systems.dbf
- inputs/building-properties/architecture.dbf
- inputs/building-properties/indoor_comfort.dbf
- inputs/building-properties/internal_loads.dbf
- inputs/building-properties/occupancy.dbf
- inputs/building-properties/supply_systems.dbf
- inputs/building-properties/schedules/B*.csv

Also, remember, the inputs/technology folder also changed and the v2.31.1 format references these possibly differently.

In [3]:
# Constants
SCENARIO_TO_MIGRATE = r"c:\Users\darthoma\Documents\CityEnergyAnalyst\projects\2.29.0\kleinalbis"
EXAMPLE_SCENARIO = r"c:\Users\darthoma\Documents\CityEnergyAnalyst\projects\working-with-databases\kleinalbis"

In [4]:
import os
import cea

## Migrate building-geometry files

In [ ]:
# let's first compare the two files (and practice reading/writing shape files)